# Telling Your Data's Story

Prospective is a tool for telling the story of your data. Today we are going to be telling a story about climate change. This story is illustrative of the various stories you can tell about _your_ data. 
Making reports, presentation, and other 'exportables' are all possible with the Perspective tool.

## Climate

We are facing numerous crises in the world today. One of the most pressing is certainly climate change. The data is in, and we need to do something. We can use the Perspective visualizer to help tell the story of climate change.

Today we are going to be looking at a few data sets and talking about how they form a compelling story about the action that we must take to save our planet! We can do it together, as long as we have a clear understanding of the problems we face.

The data is of global temperature, glacier temperatures, and our fossil fuel usage. Lets tell a story!

In [1]:
%pip install perspective-python pandas xarray netcdf4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from perspective import Table, PerspectiveWidget
import pandas as pd # For data manipulation
import xarray as xr # For parsing temperature data

## Global Temperature

The first data in our story is of global temperature over time.
The data is from [Berkeley Earth](https://berkeleyearth.org/data/) (the 5°x5° gridded global data).

This is the most straightforward visualization, a climate heat-map.

In [3]:
nc = xr.open_dataset('temperature/temp.nc')
df = nc.temperature.to_dataframe()
# This flattens the indexing in the NetCDF file.
df = pd.DataFrame(df.to_records())
df.to_feather("temperature.arrow")
temperature = Table(df)

In [4]:
# with open("temperature.arrow", 'rb') as f:
#     feather = f.read()

tw_map_layout = {
    'filter': [['integer("time")', '==', 2022]],
    'columns': ['longitude', 'latitude', '- "temperature"', None, None],
    'expressions': ['integer("time")', '- "temperature"'],
    'plugin': 'Map Scatter'
}

tw = PerspectiveWidget(temperature, server=True, **tw_map_layout)
tw

PerspectiveWidget(columns=['index', 'time', 'latitude', 'longitude', 'temperature'], server=True, theme=None)

Here we are showing anomalies of temperature. The darker red the dots, the hotter than expected. In fact, these data are quite huge! If you look at the top-bar you see we are processing ***5 MILLION*** rows of data in just a few seconds. 
This processing is done entirely in your browser, so congratulate yourself on your workhorse of a computer!

Let's look at the average temperature anomaly over time!

In [5]:
ttsw_layout = {
    'group_by': ['year'],
    'aggregates': {'temperature': 'avg'},
    'columns': ['temperature'],
    'expressions': ['// year\ninteger("time")'],
    'plugin': 'Y Line',
    'title': 'Temperature Time Series'
}

ttsw = PerspectiveWidget(temperature, server=True, **ttsw_layout)
ttsw

PerspectiveWidget(aggregates={'temperature': 'avg'}, columns=['temperature'], expressions=['// year\ninteger("…

Here we can see that the expected temperatures are growing over time. This is clearly a trend, there is not simple random anomly.

So what does rising temperature mean? It can mean hotter summers, disrupted plant cycles, and flooding of coastal cities, among others. In the context of telling a story, let's dive into one of those points: the flooding. To do this, we turn to our next dataset, glacier temperatures.

# Glaciers

This data is sourced from the 
[United States Geological Survey](https://alaska.usgs.gov/products/data.php?dataid=79).

We will start with some light data-preparation. Simply concatenating all the CSVs and outputting to the efficient Arrow format, that Perspective understands natively.

In [6]:
dfs = []
for g in ["Gulkana", "LemonCreek", "Sperry", "Wolverine"]:
    p = f"glaciers/{g}/Input_{g}_Daily_Weather.csv"
    df = pd.read_csv(p, parse_dates=['Date'])
    df["Glacier"] = g
    del df["Precipitation"]
    dfs.append(df)
weather = pd.concat(dfs, ignore_index=True)
weather.to_feather("./glacier_daily_weather.arrow")
weather = Table(weather)


It was quite easy to take these CSVs and merge them together. We can then plot this data directly into perspective.

In [7]:
# with open("glacier_daily_weather.arrow", 'rb') as f:
#     feather = f.read()

ww_layout = {
    'group_by': ['bucket("Date", \'Y\')'],
    'split_by': ['Glacier'],
    'aggregates': {'Temperature': 'avg'},
    'columns': ['Temperature'],
    'expressions': ['bucket("Date", \'Y\')'],
    'plugin': 'Y Line'
}

ww = PerspectiveWidget(weather, server=True, **ww_layout)
ww

PerspectiveWidget(aggregates={'Temperature': 'avg'}, columns=['Temperature'], expressions=['bucket("Date", \'Y…

Here, we are looking at yearly average temperatures for 4 different glaciers in Alaska. In Perspective this was done by using the "avg" aggregator, and bucketing our dates by the year. By converting the data to arrow (and doing some light [pre-processing](https://prospective.co/blog/preparing-data-to-be-used-in-prospective)), we can load and process all 75,000 rows quite quickly. The chart above is completely live, go play with the configuration!

Here, we can see that Glaciers are spending a lot of time above melting-point. What does this mean?
Melting Alaskan glaciers are believed to contribute somewhere between 10% and 30% of global sea water ([source](https://www.nature.com/articles/ngeo737/)), with other main sources including Arctic and Greenland glaciers.

Let's round out this story with a main contributor to this glacial melting: fossil fuel usage.

## Fossil Fuels

One of the main ways we can begin to transition from fossil fuels is with renewable energy.
Every year we make extremely impactful advances on the path to sustainability through 
renewable energy and better local and global programs.

But the only way we can truly prevent a climate tragedy is to divest from fossil fuels.
If we are still using fossil fuels, then we are not going to fully solve our problems.
Let's look at how much energy we use, and see if there is any good stories here.

Data from 
https://ourworldindata.org/grapher/energy-consumption-by-source-and-country

In [8]:
df = pd.read_csv("energy.csv")
# Unpivot the data given by the source.
df = df.melt(value_vars=df.columns[3:], 
             id_vars=df.columns[:3], 
             var_name="Type",
             value_name="TWh")
df.to_feather("energy.arrow")
energy = Table(df)

FileNotFoundError: [Errno 2] No such file or directory: 'energy.csv'

In just a couple lines of code, we have a great dataset regarding energy usage globally.

In [ ]:
# with open("energy.arrow", 'rb') as f:
#     feather = f.read()

# Some ExprTK Expressions to control 
# the output visualization
continent = """
// Continent
(
    "Entity" == 'Africa' or
    "Entity" == 'Asia' or
    "Entity" == 'Europe' or
    "Entity" == 'North America' or
    "Entity" == 'Oceania' or
    "Entity" == 'South America'
)
""".strip()

fossils = """
// Fossil Fuels
(
    "Type" == 'Oil consumption - TWh' or
    "Type" == 'Coal consumption - TWh' or
    "Type" == 'Gas consumption - TWh'
)
""".strip()

fw_layout = {
    'group_by': ['Year'],
    'split_by': ['Entity'],
    'filter': [['Continent', '==', True], 
               ['Fossil Fuels', '==', True]],
    'aggregates': {'TWh': 'avg'},
    'columns': ['TWh'],
    'expressions': [continent, fossils],
    'plugin': 'Y Area',
    'plugin_config': {'legend': {'left': '56px', 'top': '10px'}},
}

fw = PerspectiveWidget(energy, server=True, **fw_layout)
fw

We can see here that overall fossil fuel consumption is still rising. Of course, carbon consumption increasing means that the level of greenhouse gasses in the atmosphere will also increase.

How about renewables?


In [ ]:

layout_renewables = {
    'group_by': ['Year'],
    'split_by': ['Entity'],
    'filter': [['Continent', '==', True], ['Fossil Fuels', '==', False]],
    'aggregates': {'TWh': 'avg'},
    'columns': ['TWh'],
    'expressions': [continent, fossils],
    'plugin': 'Y Area',
    'plugin_config': {'legend': {'left': '56px', 'top': '10px'}},
}
renewables = PerspectiveWidget(energy, server=True, **layout_renewables)
renewables


It is amazing how fast we have all switched over to renewables! 

# Conclusion

Today we have gone through a couple datasets, and weaved a story throughout.

Prospective is the data visualization, data exploration, and story telling tool for all of your data.
Easily embeddable, configurable, and lightweight, your story is just waiting to be told. Start now!